In [1]:
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io

def display_rolling_correlation_viewer(returns, tickers):
    st.subheader("🔁 Rolling Correlation Viewer")
    window = st.slider("Rolling Window (days)", 20, 180, 60, key="rolling_window")
    rolling_pairs = [(a, b) for i, a in enumerate(tickers) for b in tickers[i+1:]]

    if rolling_pairs:
        pair = st.selectbox("Choose Pair", rolling_pairs, format_func=lambda x: f"{x[0]} vs {x[1]}", key="rolling_pair_main")
        t1, t2 = pair
        try:
            aligned = returns[[t1, t2]].dropna()

            with st.expander("📉 View raw return alignment for selected pair"):
                st.line_chart(aligned[[t1, t2]])
                
            st.markdown(f"### 📊 Rolling Correlation: {t1} vs {t2} ({window}-day window)")
            compute_and_plot_rolling_corr(aligned[t1], aligned[t2], window, t1, t2)

            with st.expander("🧪 Debug: Last 5 Windows Before Final Correlation Point"):
                if len(aligned) > window + 5:
                    st.dataframe(aligned.iloc[-(window + 5):])

        except Exception as e:
            st.error(f"Rolling correlation failed: {e}")

def compute_and_plot_rolling_corr(series1, series2, window, ticker1, ticker2):
    import matplotlib.pyplot as plt

    # Step 1: Join and reindex
    df_pair = pd.concat([series1, series2], axis=1)
    df_pair.columns = ["s1", "s2"]
    df_pair.index = pd.to_datetime(df_pair.index)
    full_range = pd.date_range(df_pair.index.min(), df_pair.index.max(), freq="D")
    df_pair = df_pair.reindex(full_range)
    df_pair.to_csv("example.csv")

    # Step 2: Fill missing data
    st.markdown("#### ⚙️ Missing Data Fill Method")
    interp_method = st.selectbox("Choose how to fill missing values:", ["Time Interpolation", "Forward + Backward Fill"])
    if interp_method == "Time Interpolation":
        df_pair = df_pair.interpolate(method="time")
    else:
        df_pair = df_pair.ffill().bfill()
    df_pair = df_pair.dropna()

    with st.expander("🔍 Raw Aligned Series Info"):
        st.write("Start:", df_pair.index.min())
        st.write("End:", df_pair.index.max())
        st.write("Data points:", df_pair.shape[0])
        st.write("Any NaNs?", df_pair.isnull().sum())

    if df_pair.shape[0] < window:
        st.warning(f"⚠️ Not enough data to compute {window}-day rolling correlation.")
        return

    # ✅ Step 3: Correct apply function
    def rolling_corr_func(window_df):
        x = window_df["s1"]
        y = window_df["s2"]
        if x.std() < 1e-8 or y.std() < 1e-8:
            return np.nan
        return x.corr(y)

    # Apply across the full dataframe with a rolling window
    roll_corr = df_pair.rolling(window).apply(rolling_corr_func, raw=False)["s1"]

    # Step 4: Plot
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(roll_corr.index, roll_corr, color="blue", label=f"{ticker1} vs {ticker2} ({window}d)")
    ax.axhline(0, color="gray", linestyle="--", linewidth=1)
    ax.set_title(f"Rolling Correlation ({window}-day): {ticker1} vs {ticker2}")
    ax.set_ylabel("Correlation")
    ax.set_ylim(-1.0, 1.0)
    ax.legend()
    st.pyplot(fig)